In [1]:
import numpy as np
# pip install mne
# import mne
import pandas as pd
import scipy
import matplotlib.pyplot as plt
import xml.etree.cElementTree as et
import tensorflow as tf
import librosa
from tensorflow.keras.callbacks import EarlyStopping
import time

from google.colab import drive
drive.mount('/content/drive')
rootpath='/content/drive/MyDrive/thesis/'

Mounted at /content/drive


In [2]:
def create_dataset(data, window_size,samp_rate):
    windows = []
    window_size=window_size*samp_rate
    for i in range(0, data.shape[0], window_size):
        window = data[i:i+window_size]
        if len(window) == window_size:  # Discard incomplete windows (if any)
            windows.append(window)
    return np.array(windows)

def create_mfccs(data):
    data=data.reshape(data.shape[0],)
    mfccs = librosa.feature.mfcc(y=data,n_mfcc =13,sr=500)
    delta = librosa.feature.delta(mfccs)
    delta2 = librosa.feature.delta(mfccs,order=2)
    mfccs_concat = np.concatenate((mfccs,delta,delta2))
    return mfccs_concat
def get_labels(rootpath,patient_num,event_type,window_size,ch_len):
  annotations =et.parse(rootpath+"0000"+patient_num+"-100507.rml").getroot()
  events = []
  # samp_rate=100
  for event in annotations.iter('{http://www.respironics.com/PatientStudy.xsd}Event'):
      events.append(event.attrib)
  events_df= pd.DataFrame(events)
  events_df.drop(['Machine', 'OriginatedOnDevice'], axis='columns', inplace=True)
  events_df=events_df[events_df.Family==event_type].reset_index(drop=True)
  events_df.Start=events_df.Start.astype('float64')
  events_df.Duration=events_df.Duration.astype('float64')
  events_df['end']=(events_df.Start+events_df.Duration)
  events_df['window_start']=events_df.Start//window_size
  events_df['window_end']=events_df.end//window_size
  # print(events_df)
  events_set=set(events_df.window_start)|set(events_df.window_end)
  lbls=[]
  for i in range(ch_len):
      if i in events_set:
          lbls.append(1)
      else:
          lbls.append(0)
  return lbls


def load_data(pre_train_list,downstream_list,window_size,ch4=False):
  first=True
  for num in pre_train_list:
    df = pd.read_csv(rootpath+'0000'+num+'-100507_Flow Patient-0.csv')
    df2 = pd.read_csv(rootpath+'0000'+num+'-100507_SpO2.csv')
    df3 = pd.read_csv(rootpath+'0000'+num+'-100507_Snore.csv')
    # df4 = pd.read_csv(rootpath+'0000'+num+'-100507_ECG I.csv')
    ch1_temp = create_dataset(df,window_size,100)
    ch2_temp = create_dataset(df2,window_size,1)
    ch3_temp = create_dataset(df3,window_size,500)
    # ch4_temp = create_dataset(df3,window_size,200)
    ch_len=len(ch1_temp)
    lbls1_temp = get_labels(rootpath,num,"Neuro",window_size,ch_len)
    lbls2_temp = get_labels(rootpath,num,"SpO2",window_size,ch_len)
    lbls3_temp = get_labels(rootpath,num,"Nasal",window_size,ch_len)
    lbls_temp = get_labels(rootpath,num,"Respiratory",window_size,ch_len)
    if first:
      ch1=ch1_temp
      ch2=ch2_temp
      ch3=ch3_temp
      lbls1=lbls1_temp
      lbls2=lbls2_temp
      lbls3=lbls3_temp
      lbls=lbls_temp
      first=False
    else:
      ch1=np.concatenate((ch1,ch1_temp))
      ch2=np.concatenate((ch2,ch2_temp))
      ch3=np.concatenate((ch3,ch3_temp))
      lbls1=np.concatenate((lbls1,lbls1_temp))
      lbls2=np.concatenate((lbls2,lbls2_temp))
      lbls3=np.concatenate((lbls3,lbls3_temp))
      lbls=np.concatenate((lbls,lbls_temp))

  ch3_mod = []
  for item in range(len(ch3)):
      result = create_mfccs(ch3[item]).reshape(39,30,1)
      ch3_mod.append(result)
  ch3_mod=np.array(ch3_mod)

  #DOWNSTREAM
  first=True
  window_size=30
  for num in downstream_list:
    df = pd.read_csv(rootpath+'0000'+num+'-100507_Flow Patient-0.csv')
    df2 = pd.read_csv(rootpath+'0000'+num+'-100507_SpO2.csv')
    df3 = pd.read_csv(rootpath+'0000'+num+'-100507_Snore.csv')
    ch1_temp = create_dataset(df,window_size,100)
    ch2_temp = create_dataset(df2,window_size,1)
    ch3_temp = create_dataset(df3,window_size,500)
    ch_len=len(ch1_temp)
    lbls_temp = get_labels(rootpath,num,"Respiratory",window_size,ch_len)
    if first:
      ch1_down=ch1_temp
      ch2_down=ch2_temp
      ch3_down=ch3_temp
      lbls_down=lbls_temp
      first=False
    else:
      ch1_down=np.concatenate((ch1_down,ch1_temp))
      ch2_down=np.concatenate((ch2_down,ch2_temp))
      ch3_down=np.concatenate((ch3_down,ch3_temp))
      lbls_down=np.concatenate((lbls_down,lbls_temp))

  ch3_down_mod = []
  for item in range(len(ch3_down)):
      result = create_mfccs(ch3_down[item]).reshape(39,30,1)
      ch3_down_mod.append(result)
  ch3_down_mod=np.array(ch3_down_mod)

  balance = {}
  balance["flow"]=sum(lbls1)/ch1.shape[0]
  balance["spo2"]=sum(lbls2)/ch2.shape[0]
  balance["noise"]=sum(lbls3)/ch3.shape[0]
  balance["apnea"]=sum(lbls_down)/lbls_down.shape[0]

  idx = np.random.randint(lbls.shape[0], size=lbls.shape[0])
  idx2 = np.random.randint(lbls_down.shape[0], size=lbls_down.shape[0])

  ch1=ch1[idx]
  ch2=ch2[idx]
  ch3_mod=ch3_mod[idx]
  lbls1=lbls1[idx]
  lbls2=lbls2[idx]
  lbls3=lbls3[idx]
  lbls=lbls[idx]

  ch1_down=ch1_down[idx2]
  ch2_down=ch2_down[idx2]
  ch3_down_mod=ch3_down_mod[idx2]
  lbls_down=lbls_down[idx2]

  split_ratio = int(lbls.shape[0] * 0.8)
  split_ratio_down = int(lbls_down.shape[0] * 0.8)

  return ch1,ch2,ch3_mod,lbls1,lbls2,lbls3,lbls,ch1_down,ch2_down,ch3_down_mod,lbls_down,split_ratio,split_ratio_down,balance

In [3]:
events = []
for num in ["1122","1127","1129","1131","1135"]:
  annotations =et.parse(rootpath+"00000999-100507.rml").getroot()
  for event in annotations.iter('{http://www.respironics.com/PatientStudy.xsd}Event'):
      events.append(event.attrib)
events_df= pd.DataFrame(events)
events_df.drop(['Machine', 'OriginatedOnDevice'], axis='columns', inplace=True)
events_df=events_df.groupby(['Family','Type']).agg('count')
events_df

Start  Duration
Family      Type                                 
Cardiac     LongRR                  905       905
            Tachycardia             170       170
Limb        LegMovement            1620      1620
Nasal       Snore                    65        65
Neuro       Arousal                1425      1425
Respiratory CentralApnea             60        60
            Hypopnea                240       240
            MixedApnea              100       100
            ObstructiveApnea        615       615
SpO2        RelativeDesaturation    345       345
User        ChannelFail             515       515
            Gain                    295       295

In [4]:
# first=True
# window_size=30
# # for num in ["0995","0999","1000","1006","1008","1010","1014","1016","1018","1020"]:
# for num in ["0995","0999","1000","1006","1008","1010","1014","1016","1018","1020",\
#             "1022","1024","1026","1028","1037","1039","1041","1043","1045","1057",\
#             "1059","1069","1071","1073","1082","1084","1086","1088","1089","1093",\
#             "1122","1127","1129","1131","1135","1137","1139","1143","1145","1147"]:
#             # "1095","1097","1104","1106","1108","1110","1112","1116","1118","1120"]:
#   df = pd.read_csv(rootpath+'0000'+num+'-100507_Flow Patient-0.csv')
#   df2 = pd.read_csv(rootpath+'0000'+num+'-100507_SpO2.csv')
#   df3 = pd.read_csv(rootpath+'0000'+num+'-100507_Snore.csv')
#   ch1_temp = create_dataset(df,window_size,100)
#   ch2_temp = create_dataset(df2,window_size,1)
#   ch3_temp = create_dataset(df3,window_size,500)
#   ch_len=len(ch1_temp)
#   lbls1_temp = get_labels(rootpath,num,"Neuro",window_size,ch_len)
#   lbls2_temp = get_labels(rootpath,num,"SpO2",window_size,ch_len)
#   lbls3_temp = get_labels(rootpath,num,"Nasal",window_size,ch_len)
#   lbls_temp = get_labels(rootpath,num,"Respiratory",window_size,ch_len)
#   if first:
#     ch1=ch1_temp
#     ch2=ch2_temp
#     ch3=ch3_temp
#     lbls1=lbls1_temp
#     lbls2=lbls2_temp
#     lbls3=lbls3_temp
#     lbls=lbls_temp
#     first=False
#   else:
#     ch1=np.concatenate((ch1,ch1_temp))
#     ch2=np.concatenate((ch2,ch2_temp))
#     ch3=np.concatenate((ch3,ch3_temp))
#     lbls1=np.concatenate((lbls1,lbls1_temp))
#     lbls2=np.concatenate((lbls2,lbls2_temp))
#     lbls3=np.concatenate((lbls3,lbls3_temp))
#     lbls=np.concatenate((lbls,lbls_temp))

# ch3_mod = []
# for item in range(len(ch3)):
#     result = create_mfccs(ch3[item]).reshape(39,30,1)
#     ch3_mod.append(result)
# #     plt.figure(figsize=(25,10))
# #     librosa.display.specshow(result,x_axis='time',sr=500)
# #     plt.colorbar(format='%+2f')
# ch3_mod=np.array(ch3_mod)



In [5]:
# first=True
# window_size=30
# # for num in ["1122","1127","1129","1131","1135","1137","1139","1143","1145","1147"]:
# for num in ["1095","1097","1104","1106","1108","1110","1112","1116","1118","1120"]:
# # for num in ["1122","1127","1129","1131","1135"]:
#   df = pd.read_csv(rootpath+'0000'+num+'-100507_Flow Patient-0.csv')
#   df2 = pd.read_csv(rootpath+'0000'+num+'-100507_SpO2.csv')
#   df3 = pd.read_csv(rootpath+'0000'+num+'-100507_Snore.csv')
#   ch1_temp = create_dataset(df,window_size,100)
#   ch2_temp = create_dataset(df2,window_size,1)
#   ch3_temp = create_dataset(df3,window_size,500)
#   ch_len=len(ch1_temp)
#   lbls_temp = get_labels(rootpath,num,"Respiratory",window_size,ch_len)
#   if first:
#     ch1_down=ch1_temp
#     ch2_down=ch2_temp
#     ch3_down=ch3_temp
#     lbls_down=lbls_temp
#     first=False
#   else:
#     ch1_down=np.concatenate((ch1_down,ch1_temp))
#     ch2_down=np.concatenate((ch2_down,ch2_temp))
#     ch3_down=np.concatenate((ch3_down,ch3_temp))
#     lbls_down=np.concatenate((lbls_down,lbls_temp))

# ch3_down_mod = []
# for item in range(len(ch3_down)):
#     result = create_mfccs(ch3_down[item]).reshape(39,30,1)
#     ch3_down_mod.append(result)
# #     plt.figure(figsize=(25,10))
# #     librosa.display.specshow(result,x_axis='time',sr=500)
# #     plt.colorbar(format='%+2f')
# ch3_down_mod=np.array(ch3_down_mod)

In [6]:
# print(ch1.shape,ch2.shape,ch3_mod.shape,lbls.shape)
# print(lbls_down.shape)

In [7]:
# print(sum(lbls1),sum(lbls2),sum(lbls3),sum(lbls))
# print(sum(lbls_down))

In [8]:
# idx = np.random.randint(lbls.shape[0], size=lbls.shape[0])
# idx2 = np.random.randint(lbls_down.shape[0], size=lbls_down.shape[0])

In [9]:
# ch1=ch1[idx]
# ch2=ch2[idx]
# ch3_mod=ch3_mod[idx]
# lbls1=lbls1[idx]
# lbls2=lbls2[idx]
# lbls3=lbls3[idx]
# lbls=lbls[idx]

# ch1_down=ch1_down[idx2]
# ch2_down=ch2_down[idx2]
# ch3_down_mod=ch3_down_mod[idx2]
# lbls_down=lbls_down[idx2]

In [10]:
# split_ratio = int(lbls.shape[0] * 0.8)
# split_ratio_down = int(lbls_down.shape[0] * 0.8)

In [11]:
def encoder_1d(input_shape, modality_name,filters,code_size,l2_rate):
    initializer = tf.keras.initializers.RandomNormal(mean=0., stddev=1.)
    input = tf.keras.layers.Input(input_shape)
    x = tf.keras.layers.Conv1D(filters=2 * filters,
                               kernel_size=10,
                               activation="linear",
                               padding="same",
                               strides=1,
                               kernel_regularizer=tf.keras.regularizers.l2(l2_rate),
                               kernel_initializer=initializer)(input)

    x = tf.keras.layers.LayerNormalization()(x)
    x = tf.keras.layers.PReLU(shared_axes=[1])(x)

    x = tf.keras.layers.Conv1D(filters=filters,
                               kernel_size=8,
                               activation="linear",
                               padding="same",
                               strides=1,
                               kernel_regularizer=tf.keras.regularizers.l2(l2_rate),
                               kernel_initializer=initializer)(x)
    x = tf.keras.layers.LayerNormalization()(x)
    x = tf.keras.layers.PReLU(shared_axes=[1])(x)

    x = tf.keras.layers.Conv1D(filters=code_size,
                               kernel_size=4,
                               activation="linear",
                               padding="same",
                               strides=1,
                               kernel_regularizer=tf.keras.regularizers.l2(l2_rate),
                               kernel_initializer=initializer)(x)
    # output = tf.keras.layers.LayerNormalization()(x)
    x = tf.keras.layers.PReLU(shared_axes=[1])(x)
    output = tf.keras.layers.BatchNormalization(epsilon=1e-6)(x)

    return tf.keras.models.Model(input, output, name=modality_name)

def encoder_2d(input_shape, modality_name,filters):
    input = tf.keras.layers.Input(input_shape) #input_shape=(height, width, channels)
    # Add a 2D convolutional layer with 32 filters, 5x5 kernel size, and 'relu' activation
    x = tf.keras.layers.Conv2D(filters, (5, 5), activation='relu')(input)

    # Add a 2D max pooling layer with 2x2 pool size
    x=tf.keras.layers.MaxPooling2D(pool_size=(2, 2))(x)

    # Add a dropout layer with a dropout rate of 0.25 (adjust as needed)
    x=tf.keras.layers.Dropout(0.25)(x)

    # Add a 2D convolutional layer with 32 filters, 5x5 kernel size, and 'relu' activation
    x = tf.keras.layers.Conv2D(filters*2, (4, 4), activation='relu')(x)

    # Add a 2D max pooling layer with 2x2 pool size
    x=tf.keras.layers.MaxPooling2D(pool_size=(2, 2))(x)

    # Add a dropout layer with a dropout rate of 0.25 (adjust as needed)
    x=tf.keras.layers.Dropout(0.25)(x)

    # Add a 2D convolutional layer with 32 filters, 5x5 kernel size, and 'relu' activation
    x = tf.keras.layers.Conv2D(filters*3, (3, 3), activation='relu')(x)

    # Add a dropout layer with a dropout rate of 0.25 (adjust as needed)
    output=tf.keras.layers.Dropout(0.25)(x)

    return tf.keras.models.Model(input, output, name=modality_name)

In [12]:
# modality specific encoders
ch1,ch2,ch3_mod,lbls1,lbls2,lbls3,lbls,ch1_down,ch2_down,ch3_down_mod,lbls_down,split_ratio,split_ratio_down,balance=load_data(['0995',"0999"],['0995',"0999"],30)
mod_encoder = []
mod_preclass= []
mod_input = []

input_shape = (ch1.shape[1], ch1.shape[2])
encoder = encoder_1d(input_shape,
                                  modality_name='flow',
                                  filters=24,
                                  code_size=256,
                                  l2_rate=1e-4)


mod_input.append(tf.keras.layers.Input(shape=input_shape))

x_a = encoder(mod_input[-1])
x_a = tf.keras.layers.GlobalMaxPooling1D()(x_a)
x_a = tf.keras.layers.Dense(256, activation="linear")(x_a)
mod_encoder.append(x_a)
x_a = tf.keras.layers.Dense(128, activation="relu")(x_a)
x_a = tf.keras.layers.Dense(1, activation="sigmoid")(x_a)
mod_preclass.append(x_a)

input_shape = (ch2.shape[1], ch2.shape[2])
encoder = encoder_1d(input_shape,
                                  modality_name='spo2',
                                  filters=24,
                                  code_size=256,
                                  l2_rate=1e-4)

mod_input.append(tf.keras.layers.Input(shape=input_shape))

x_a = encoder(mod_input[-1])
x_a = tf.keras.layers.GlobalMaxPooling1D()(x_a)
x_a = tf.keras.layers.Dense(256, activation="linear")(x_a)
mod_encoder.append(x_a)
x_a = tf.keras.layers.Dense(128, activation="relu")(x_a)
x_a = tf.keras.layers.Dense(1, activation="sigmoid")(x_a)
mod_preclass.append(x_a)


input_shape = (ch3_mod.shape[1], ch3_mod.shape[2],ch3_mod.shape[3])
encoder = encoder_2d(input_shape,
                                  modality_name='snore',
                                  filters=36)

mod_input.append(tf.keras.layers.Input(shape=input_shape))

x_a = encoder(mod_input[-1])
x_a = tf.keras.layers.GlobalMaxPooling2D()(x_a)
x_a = tf.keras.layers.Dense(256, activation="linear")(x_a)
mod_encoder.append(x_a)
x_a = tf.keras.layers.Dense(128, activation="relu")(x_a)
x_a = tf.keras.layers.Dense(1, activation="sigmoid")(x_a)
mod_preclass.append(x_a)


embedding_model = tf.keras.Model(mod_input, mod_encoder)
preclass_model = tf.keras.Model(mod_input, mod_preclass)

embedding_model.summary()
preclass_model.summary()

/usr/local/lib/python3.10/dist-packages/keras/src/initializers/initializers.py:120: UserWarning: The initializer RandomNormal is unseeded and being called multiple times, which will return identical values each time (even if the initializer is unseeded). Please update your code to provide a seed to the initializer, or avoid using the same initializer instance more than once.
  warnings.warn(


Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_2 (InputLayer)        [(None, 3000, 1)]            0         []                            
                                                                                                  
 input_4 (InputLayer)        [(None, 30, 1)]              0         []                            
                                                                                                  
 input_6 (InputLayer)        [(None, 39, 30, 1)]          0         []                            
                                                                                                  
 flow (Functional)           (None, 3000, 256)            36096     ['input_2[0][0]']             
                                                                                              

In [13]:
earlyStopping_class = EarlyStopping(monitor="loss",min_delta=0.00009,patience=6,verbose=2,mode="auto",restore_best_weights=True)

earlyStopping_ssl = EarlyStopping(monitor="loss",
                                      min_delta=0.00009,
                                      patience=5,
                                      verbose=1,
                                      mode="min",
                                      restore_best_weights=True
                                      )


# from keras.callbacks import Callback

# class CustomCallback(Callback):
#     def on_epoch_end(self, epoch, logs=None):
#         if epoch % 10 == 0:  # Check if it's the 10th epoch
#             loss, acc = self.model.evaluate(x_val, y_val, verbose=0)
#             print(f'\nEvaluation loss and accuracy after epoch {epoch}: {loss:.4f}, {acc:.4f}\n')

## PRE-Classification method

In [14]:
preclass_model.compile(
    optimizer='adam',
    loss=["binary_crossentropy","binary_crossentropy","binary_crossentropy"],
    metrics=[tf.keras.metrics.BinaryAccuracy(),tf.keras.metrics.Precision(),tf.keras.metrics.Recall()],
)


## Contrastive loss

In [15]:
class customLoss:
  def __init__(self,temperature=0.5,lambd = 3.9e-3,scale_loss= 1/32):
    self.temperature = temperature
    self.lambd = lambd
    self.scale_loss = scale_loss
  def get_loss(self,loss_type):
    if loss_type=='cocoa':
      def cocoa_loss(ytrue, ypred):
        # temperature = 0.5
        # lambd=3.9e-3
        # scale_loss=1/32
        batch_size, dim_size = ypred.shape[1], ypred.shape[0]
        # print(ypred.shape)
        # Positive Pairs
        pos_error = []
        for i in range(batch_size):
            sim = tf.linalg.matmul(ypred[:, i, :], ypred[:, i, :], transpose_b=True)
            sim = tf.subtract(tf.ones([dim_size, dim_size], dtype=tf.dtypes.float32), sim)
            sim = tf.exp(sim/self.temperature)
            pos_error.append(tf.reduce_mean(sim))
        # Negative pairs
        neg_error = 0
        for i in range(dim_size):
            sim = tf.cast(tf.linalg.matmul(ypred[i], ypred[i], transpose_b=True), dtype=tf.dtypes.float32)
            sim = tf.exp(sim /self.temperature)
            # sim = tf.add(sim, tf.ones([batch_size, batch_size]))
            tri_mask = np.ones(batch_size ** 2, dtype=bool).reshape(batch_size, batch_size)
            tri_mask[np.diag_indices(batch_size)] = False
            off_diag_sim = tf.reshape(tf.boolean_mask(sim, tri_mask), [batch_size, batch_size - 1])
            neg_error += (tf.reduce_mean(off_diag_sim, axis=-1))

        error = tf.multiply(tf.reduce_sum(pos_error),self.scale_loss) + self.lambd * tf.reduce_sum(neg_error)

        return error
      return cocoa_loss
# ------------------------------------------------------------------------- #
class DotProduct(tf.keras.layers.Layer):
    def call(self, x, y):
        x = tf.nn.l2_normalize(x, axis=-1)
        y = tf.nn.l2_normalize(y, axis=-1)
        return tf.linalg.matmul(x, y, transpose_b=True)


# ------------------------------------------------------------------------- #
class ContrastiveModel(tf.keras.Model):
    def __init__(self, embedding_model, loss_fn, temperature=1.0, **kwargs):
        super().__init__()
        self.embedding_model = embedding_model
        self._temperature = temperature
        self._similarity_layer = DotProduct()
        self._lossfn = loss_fn

    def train_step(self, data):
        with tf.GradientTape() as tape:
            modality_embeddings = self.embedding_model(data, training=True)

            sparse_labels = tf.range(tf.shape(modality_embeddings[0])[0])

            pred = modality_embeddings
            pred = tf.nn.l2_normalize(tf.stack(pred), axis=-1)

            loss = self.compiled_loss(sparse_labels, pred)
            loss += sum(self.losses)

        trainable_vars = self.trainable_variables
        gradients = tape.gradient(loss, trainable_vars)
        self.optimizer.apply_gradients(zip(gradients, trainable_vars))
        return {m.name: m.result() for m in self.metrics}

    def call(self, input):
        return self.embedding_model(input)


In [16]:
cocoa_loss = customLoss().get_loss('cocoa')
ssl = ContrastiveModel(embedding_model, cocoa_loss)

# init_weights = ssl.get_weights()

## add the base model

In [17]:
base_model = ssl.embedding_model
# for layer in base_model.layers:
#     print(layer)
#     layer.trainable = False
input_x = base_model.input
xi = base_model(input_x)
x = tf.keras.layers.Concatenate()(xi)
x = tf.keras.layers.Flatten()(x)

x = tf.keras.layers.Dense(128, activation="relu", kernel_regularizer=tf.keras.regularizers.L1(0.005))(x)

classifier_model = tf.keras.layers.Dense(1,activation="sigmoid",kernel_regularizer=tf.keras.regularizers.L1(0.005),)(x)
c_model = tf.keras.Model(input_x, classifier_model)
c_model.compile(
    optimizer='adam',
    loss="binary_crossentropy",
    metrics=[tf.keras.metrics.BinaryAccuracy(),tf.keras.metrics.Precision(),tf.keras.metrics.Recall()],
)
init_weights = c_model.get_weights()

In [18]:
c_model.summary()

Model: "model_2"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_2 (InputLayer)        [(None, 3000, 1)]            0         []                            
                                                                                                  
 input_4 (InputLayer)        [(None, 30, 1)]              0         []                            
                                                                                                  
 input_6 (InputLayer)        [(None, 39, 30, 1)]          0         []                            
                                                                                                  
 model (Functional)          [(None, 256),                344252    ['input_2[0][0]',             
                              (None, 256),                           'input_4[0][0]',       

In [19]:
complete_list = np.array(["0995","0999","1000","1006","1008","1010","1014","1016","1018","1020",\
            "1022","1024","1026","1028","1037","1039","1041","1043","1045","1057",\
            "1059","1069","1071","1073","1082","1084","1086","1088","1089","1093",\
            "1095","1097","1104","1106","1108","1110","1112","1116","1118","1120",\
            "1122","1127","1129","1131","1135","1137","1139","1143","1145","1147",\
            "1151","1153","1155","1157","1161","1163","1169","1171","1172","1174",\
            "1176","1178","1182","1186","1191","1193","1195","1197","1198","1200"])



In [ ]:
final_hist = {}
run_data={}
for item in ['flow_balance','spo2_balance','noise_balance','apnea_balance','pre_train','downstream',\
             'test_accuracy1','test_precision1','test_recall1','test_fscore1',\
              'test_accuracy2','test_precision2','test_recall2','test_fscore2',\
              'test_accuracy3','test_precision3','test_recall3','test_fscore3']:
  run_data[item]=[]

for run in range(8):

  idx = np.random.randint(len(complete_list), size=len(complete_list))
  complete_list=complete_list[idx]
  pre_train_list = complete_list[:-10]
  downstream_list = complete_list[-10:]
  window_size=30
  ch1,ch2,ch3_mod,lbls1,lbls2,lbls3,lbls,ch1_down,ch2_down,ch3_down_mod,lbls_down,split_ratio,split_ratio_down,balance=load_data(pre_train_list,downstream_list,window_size)
  print(balance)
  run_data['pre_train'].append(pre_train_list)
  run_data['downstream'].append(downstream_list)

  run_data['flow_balance'].append(balance['flow'])
  run_data['spo2_balance'].append(balance['spo2'])
  run_data['noise_balance'].append(balance['noise'])
  run_data['apnea_balance'].append(balance['apnea'])
  #Case 1 - full network
  print(f"This is run {run}. Case 1")
  start = time.time()

  #pre class train
  batch_size=300
  train,val=tf.data.Dataset.from_tensor_slices((ch1[0:split_ratio], ch2[0:split_ratio], ch3_mod[0:split_ratio])),\
            tf.data.Dataset.from_tensor_slices((ch1[split_ratio:], ch2[split_ratio:], ch3_mod[split_ratio:]))
  train_lbl,val_lbl =  tf.data.Dataset.from_tensor_slices((lbls1[0:split_ratio], lbls2[0:split_ratio], lbls3[0:split_ratio])),\
                      tf.data.Dataset.from_tensor_slices((lbls1[split_ratio:], lbls2[split_ratio:], lbls3[split_ratio:]))
  train=tf.data.Dataset.zip(train,train_lbl)
  val=tf.data.Dataset.zip(val,val_lbl)
  training= train.batch(batch_size, drop_remainder=True).prefetch(tf.data.experimental.AUTOTUNE)
  validation = val.batch(batch_size, drop_remainder=True).prefetch(tf.data.experimental.AUTOTUNE)
  preclass_hist = preclass_model.fit(training,epochs=50,verbose =0,callbacks=[earlyStopping_class])

  # CL
  temperature = 0.4
  lambd=3.9e-1
  scale_loss=1/32
  batch_size=8
  train=tf.data.Dataset.from_tensor_slices((ch1, ch2, ch3_mod))
  X_train= train.batch(batch_size, drop_remainder=True).prefetch(tf.data.experimental.AUTOTUNE)
  cocoa_loss = customLoss(temperature,lambd,scale_loss).get_loss('cocoa')
  ssl.compile(optimizer='adam', loss=cocoa_loss, metrics=[tf.keras.metrics.SparseCategoricalAccuracy()])
  ssl_hist = ssl.fit(X_train, epochs=30,verbose =0,callbacks=[earlyStopping_ssl])

  # downstream
  batch_size=300
  train,val=tf.data.Dataset.from_tensor_slices((ch1_down[0:split_ratio_down], ch2_down[0:split_ratio_down], ch3_down_mod[0:split_ratio_down])),\
            tf.data.Dataset.from_tensor_slices((ch1_down[split_ratio_down:], ch2_down[split_ratio_down:], ch3_down_mod[split_ratio_down:]))
  train_lbl,val_lbl =  tf.data.Dataset.from_tensor_slices(lbls_down[0:split_ratio_down]),\
                      tf.data.Dataset.from_tensor_slices(lbls_down[split_ratio_down:])
  train=tf.data.Dataset.zip(train,train_lbl)
  val=tf.data.Dataset.zip(val,val_lbl)
  training= train.batch(batch_size, drop_remainder=True).prefetch(tf.data.experimental.AUTOTUNE)
  validation = val.batch(batch_size, drop_remainder=True).prefetch(tf.data.experimental.AUTOTUNE)
  class_hist = c_model.fit(training, epochs=80,verbose =0,callbacks=[earlyStopping_class])

  test_loss, test_accuracy,test_precision,test_recall = c_model.evaluate(validation)
  print(f'Test fscore: {2*test_precision*test_recall/(test_precision+test_recall)}')
  run_data['test_accuracy1'].append(test_accuracy)
  run_data['test_precision1'].append(test_precision)
  run_data['test_recall1'].append(test_recall)
  run_data['test_fscore1'].append(2*test_precision*test_recall/(test_precision+test_recall))

  final_hist['case1.'+str(run)]=class_hist.history
  c_model.set_weights(init_weights)

  end = time.time()
  print(f'Time took for Case 1: {end - start}s')
  ## Case 2 - only CL + downstream ######################
  print(f"This is run {run}. Case 2")
  start = time.time()
  # CL
  batch_size=8
  train=tf.data.Dataset.from_tensor_slices((ch1, ch2, ch3_mod))
  X_train= train.batch(batch_size, drop_remainder=True).prefetch(tf.data.experimental.AUTOTUNE)
  cocoa_loss = customLoss(temperature,lambd,scale_loss).get_loss('cocoa')
  ssl.compile(optimizer='adam', loss=cocoa_loss, metrics=[tf.keras.metrics.SparseCategoricalAccuracy()])
  ssl_hist = ssl.fit(X_train, epochs=30,verbose =0,callbacks=[earlyStopping_ssl])

  # downstream
  batch_size=300
  train,val=tf.data.Dataset.from_tensor_slices((ch1_down[0:split_ratio_down], ch2_down[0:split_ratio_down], ch3_down_mod[0:split_ratio_down])),\
            tf.data.Dataset.from_tensor_slices((ch1_down[split_ratio_down:], ch2_down[split_ratio_down:], ch3_down_mod[split_ratio_down:]))
  train_lbl,val_lbl =  tf.data.Dataset.from_tensor_slices(lbls_down[0:split_ratio_down]),\
                      tf.data.Dataset.from_tensor_slices(lbls_down[split_ratio_down:])
  train=tf.data.Dataset.zip(train,train_lbl)
  val=tf.data.Dataset.zip(val,val_lbl)
  training= train.batch(batch_size, drop_remainder=True).prefetch(tf.data.experimental.AUTOTUNE)
  validation = val.batch(batch_size, drop_remainder=True).prefetch(tf.data.experimental.AUTOTUNE)
  class_hist = c_model.fit(training, epochs=80,verbose =0,callbacks=[earlyStopping_class])

  test_loss, test_accuracy,test_precision,test_recall = c_model.evaluate(validation)
  print(f'Test fscore: {2*test_precision*test_recall/(test_precision+test_recall)}')

  run_data['test_accuracy2'].append(test_accuracy)
  run_data['test_precision2'].append(test_precision)
  run_data['test_recall2'].append(test_recall)
  run_data['test_fscore2'].append(2*test_precision*test_recall/(test_precision+test_recall))
  final_hist['case2.'+str(run)]=class_hist.history

  c_model.set_weights(init_weights)
  end = time.time()
  print(f'Time took for Case 2: {end - start}s')

  ## Case 3 - only downstream #############################
  print(f"This is run {run}. Case 3")
  start = time.time()
  # downstream
  batch_size=300
  train,val=tf.data.Dataset.from_tensor_slices((ch1_down[0:split_ratio_down], ch2_down[0:split_ratio_down], ch3_down_mod[0:split_ratio_down])),\
            tf.data.Dataset.from_tensor_slices((ch1_down[split_ratio_down:], ch2_down[split_ratio_down:], ch3_down_mod[split_ratio_down:]))
  train_lbl,val_lbl =  tf.data.Dataset.from_tensor_slices(lbls_down[0:split_ratio_down]),\
                      tf.data.Dataset.from_tensor_slices(lbls_down[split_ratio_down:])
  train=tf.data.Dataset.zip(train,train_lbl)
  val=tf.data.Dataset.zip(val,val_lbl)
  training= train.batch(batch_size, drop_remainder=True).prefetch(tf.data.experimental.AUTOTUNE)
  validation = val.batch(batch_size, drop_remainder=True).prefetch(tf.data.experimental.AUTOTUNE)
  class_hist = c_model.fit(training, epochs=80,verbose =0,callbacks=[earlyStopping_class])

  test_loss, test_accuracy,test_precision,test_recall = c_model.evaluate(validation)
  print(f'Test fscore: {2*test_precision*test_recall/(test_precision+test_recall)}')
  run_data['test_accuracy3'].append(test_accuracy)
  run_data['test_precision3'].append(test_precision)
  run_data['test_recall3'].append(test_recall)
  run_data['test_fscore3'].append(2*test_precision*test_recall/(test_precision+test_recall))
  final_hist['case3.'+str(run)]=class_hist.history

  end = time.time()
  print(f'Time took for Case 3: {end - start}s')

  c_model.set_weights(init_weights)

In [ ]:
results = pd.DataFrame(run_data)
saved=False
num=0
while not saved:
  try:
    saved=True
    results.to_csv(rootpath+f'results_{num}.csv', mode='x')
  except:
    num+=1
    saved=False
results

In [ ]:
saved=False
train_results = pd.DataFrame(final_hist)
num=0
while not saved:
  try:
    saved=True
    train_results.to_csv(rootpath+f'train_results_{num}.csv', mode='x')
  except:
    num+=1
    saved=False

train_results

# import json

# with open(rootpath+f'train_results_{num}.txt', 'w') as file:
#   file.write(json.dumps(final_hist)) # use `json.loads` to do the reverse